In [3]:
import cv2
import mediapipe as mp
import numpy as np
import pickle
from sklearn.ensemble import RandomForestClassifier

# --- CONFIGURACIÓN ---
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.5)

data = []
labels = []

# Diccionario de etiquetas para tu referencia
gestos_map = {0: "NEUTRO", 1: "PUÑO", 2: "PALMA", 3: "VICTORIA"}

cap = cv2.VideoCapture(0)
print("--- MODO ENTRENAMIENTO ---")
print("Instrucciones: Haz el gesto y mantén presionada la tecla correspondiente para guardar datos.")
print("0: Neutro | 1: Puño | 2: Palma | 3: Victoria | Q: Salir y Entrenar")

while True:
    ret, frame = cap.read()
    if not ret: break
    
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)
    
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # Dibujar esqueleto
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            
            # --- EXTRACCIÓN Y NORMALIZACIÓN DE DATOS ---
            landmark_list = []
            for lm in hand_landmarks.landmark:
                landmark_list.append(lm.x)
                landmark_list.append(lm.y)
            
            # Teclas para etiquetar
            k = cv2.waitKey(1)
            if k == ord('0'): # Neutro
                data.append(landmark_list)
                labels.append(0)
                print("Guardado: Neutro")
            elif k == ord('1'): # Puño
                data.append(landmark_list)
                labels.append(1)
                print("Guardado: Puño")
            elif k == ord('2'): # Palma
                data.append(landmark_list)
                labels.append(2)
                print("Guardado: Palma")
            elif k == ord('3'): # Victoria
                data.append(landmark_list)
                labels.append(3)
                print("Guardado: Victoria")
            elif k == ord('q'):
                break

    cv2.imshow('Entrenador de Gestos', frame)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# --- ENTRENAMIENTO DEL MODELO ---
if len(data) > 0:
    print(f"Entrenando con {len(data)} muestras...")
    data = np.array(data)
    labels = np.array(labels)
    
    model = RandomForestClassifier()
    model.fit(data, labels)
    
    with open('modelo_gestos.pkl', 'wb') as f:
        pickle.dump(model, f)
    print("¡Modelo guardado exitosamente como 'modelo_gestos.pkl'!")
else:
    print("No se guardaron datos. No se creó el modelo.")

--- MODO ENTRENAMIENTO ---
Instrucciones: Haz el gesto y mantén presionada la tecla correspondiente para guardar datos.
0: Neutro | 1: Puño | 2: Palma | 3: Victoria | Q: Salir y Entrenar


c:\Users\Laura\anaconda3\envs\VC_Final\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Guardado: Puño
Guardado: Palma
Guardado: Victoria
Entrenando con 3 muestras...
¡Modelo guardado exitosamente como 'modelo_gestos.pkl'!


In [4]:
import cv2
import mediapipe as mp
import pickle
import numpy as np
import random
import time
import pyttsx3
import threading

# --- CONFIGURACIÓN INICIAL ---
try:
    with open('modelo_gestos.pkl', 'rb') as f:
        model = pickle.load(f)
except FileNotFoundError:
    print("ERROR: Ejecuta primero entrenador.py")
    exit()

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.7)

# Mapeo inverso (Debe coincidir con el entrenador)
GESTOS = {0: "NEUTRO", 1: "PUÑO", 2: "PALMA", 3: "VICTORIA"}
NOMBRES_TRAMPA = ["Modesto", "Pepe", "La Profe"]

# Motor de voz (en un hilo separado para no congelar el video)
engine = pyttsx3.init()
def hablar(texto):
    def _speak():
        try:
            engine.say(texto)
            engine.runAndWait()
        except: pass
    threading.Thread(target=_speak).start()

# --- VARIABLES DE ESTADO DEL JUEGO ---
ESTADO = "INICIO" # INICIO, MOSTRAR_SECUENCIA, INPUT_JUGADOR, GAME_OVER
secuencia = []
paso_usuario = 0
tiempo_limite = 0
mensaje_pantalla = "Presiona 'S' para empezar"
subtitulo = ""
score = 0
ultimo_gesto_detectado = "NEUTRO"
conteo_frames_gesto = 0 # Para evitar falsos positivos
gesto_confirmado = "NEUTRO"

# --- BUCLE PRINCIPAL ---
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret: break
    
    # 1. Procesamiento de Visión
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)
    
    prediccion_actual = "NEUTRO"
    
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            
            # Extraer puntos para el modelo
            aux = []
            for lm in hand_landmarks.landmark:
                aux.append(lm.x)
                aux.append(lm.y)
            
            # Predecir
            try:
                clase = model.predict([aux])[0]
                prediccion_actual = GESTOS[clase]
            except: pass
            
    # Estabilizador de gesto (Debounce): El gesto debe mantenerse 5 frames para ser válido
    if prediccion_actual == ultimo_gesto_detectado:
        conteo_frames_gesto += 1
    else:
        conteo_frames_gesto = 0
        ultimo_gesto_detectado = prediccion_actual
        
    if conteo_frames_gesto > 5:
        gesto_confirmado = prediccion_actual

    # --- MÁQUINA DE ESTADOS DEL JUEGO ---
    
    if ESTADO == "INICIO":
        cv2.putText(frame, "SIMON VISUAL", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.putText(frame, "Pulsa 'S' para iniciar", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        
        k = cv2.waitKey(1)
        if k == ord('s'):
            ESTADO = "GENERAR_RONDA"
            score = 0
            secuencia = []

    elif ESTADO == "GENERAR_RONDA":
        # Añadir un nuevo gesto a la secuencia
        nuevo_gesto = random.choice(["PUÑO", "PALMA", "VICTORIA"])
        secuencia.append(nuevo_gesto)
        
        # Determinar si es trampa (Solo importa en el momento de decirlo, aquí simplificamos)
        # En esta versión: Simón dicta la secuencia completa y tú la repites.
        # La trampa la aplicaremos al "dar la orden".
        
        ESTADO = "MOSTRAR_SECUENCIA"
        indice_demo = 0
        tiempo_inicio_demo = time.time()
        es_trampa_ronda = False
        nombre_emisor = "Simon"
        
        # Probabilidad de trampa (30%)
        if random.random() > 0.7:
            es_trampa_ronda = True
            nombre_emisor = random.choice(NOMBRES_TRAMPA)

        texto_orden = f"{nombre_emisor} dice..."
        hablar(texto_orden)
        mensaje_pantalla = texto_orden

    elif ESTADO == "MOSTRAR_SECUENCIA":
        # Mostrar la secuencia al usuario visualmente
        tiempo_actual = time.time()
        
        if tiempo_actual - tiempo_inicio_demo > 2.0: # Cada 2 segundos cambia gesto
            if indice_demo < len(secuencia):
                gesto_mostrar = secuencia[indice_demo]
                mensaje_pantalla = f"Gesto {indice_demo + 1}: {gesto_mostrar}"
                
                # Visualización del gesto esperado (Texto grande)
                cv2.putText(frame, gesto_mostrar, (200, 200), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 255), 3)
                hablar(gesto_mostrar)
                
                indice_demo += 1
                tiempo_inicio_demo = time.time()
            else:
                # Fin de mostrar secuencia
                ESTADO = "INPUT_JUGADOR"
                paso_usuario = 0
                mensaje_pantalla = "¡TU TURNO!"
                hablar("Tu turno")
                tiempo_limite = time.time() + (5 * len(secuencia)) # 5 seg por gesto

    elif ESTADO == "INPUT_JUGADOR":
        tiempo_restante = int(tiempo_limite - time.time())
        cv2.putText(frame, f"Tiempo: {tiempo_restante}", (450, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        
        gesto_objetivo = secuencia[paso_usuario]
        subtitulo = f"Esperando: {gesto_objetivo} ({paso_usuario + 1}/{len(secuencia)})"

        # Lógica de TRAMPA
        if es_trampa_ronda:
            mensaje_pantalla = "¡ERA TRAMPA! NO HAGAS NADA"
            # Si el usuario hace CUALQUIER gesto que no sea NEUTRO, pierde
            if gesto_confirmado != "NEUTRO":
                ESTADO = "GAME_OVER"
                motivo = f"Caiste en la trampa de {nombre_emisor}"
            
            # Si aguanta 3 segundos sin hacer nada, gana la ronda
            if time.time() > tiempo_limite - (5 * len(secuencia)) + 3:
                mensaje_pantalla = "¡Bien hecho! No caiste."
                hablar("Bien hecho")
                ESTADO = "GENERAR_RONDA" # Siguiente nivel (se añade otro gesto)
                # Nota: En trampa no se añade gesto a la secuencia para no complicarlo demasiado, 
                # o se puede vaciar la secuencia. Aquí simplemente pasamos ronda.
                
        else: # Juego normal (Simón dice)
            if tiempo_restante <= 0:
                ESTADO = "GAME_OVER"
                motivo = "Tiempo agotado"
            
            # Validar gesto
            if gesto_confirmado == gesto_objetivo:
                # Acierto
                paso_usuario += 1
                hablar("Correcto")
                mensaje_pantalla = "¡BIEN!"
                # Esperar un poco para que no detecte el mismo gesto dos veces instantáneamente
                gesto_confirmado = "NEUTRO" 
                time.sleep(0.5) 
                
                if paso_usuario >= len(secuencia):
                    score += 100
                    ESTADO = "GENERAR_RONDA"
            
            elif gesto_confirmado != "NEUTRO" and gesto_confirmado != gesto_objetivo:
                # Error (Hizo el gesto incorrecto)
                ESTADO = "GAME_OVER"
                motivo = f"Hiciste {gesto_confirmado}, era {gesto_objetivo}"

    elif ESTADO == "GAME_OVER":
        cv2.putText(frame, "GAME OVER", (150, 200), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 4)
        cv2.putText(frame, f"Puntuacion: {score}", (180, 260), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        cv2.putText(frame, motivo, (50, 300), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (200, 200, 200), 2)
        cv2.putText(frame, "Presiona 'R' para reiniciar", (120, 350), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)
        
        if cv2.waitKey(1) == ord('r'):
            ESTADO = "INICIO"

    # --- INTERFAZ GRAFICA ---
    # Barra inferior de estado
    cv2.rectangle(frame, (0, 400), (640, 480), (0, 0, 0), -1)
    cv2.putText(frame, mensaje_pantalla, (20, 430), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
    cv2.putText(frame, subtitulo, (20, 460), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (150, 150, 150), 1)
    
    # Mostrar qué está detectando la IA ahora mismo
    cv2.putText(frame, f"IA ve: {gesto_confirmado}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0) if gesto_confirmado != "NEUTRO" else (0, 0, 255), 2)

    cv2.imshow('Juego Simon IA', frame)
    
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

c:\Users\Laura\anaconda3\envs\VC_Final\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
c:\Users\Laura\anaconda3\envs\VC_Final\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
c:\Users\Laura\anaconda3\envs\VC_Final\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
c:\User